In [1]:
!pip install -U google-genai python-dotenv pandas tabulate

In [2]:
from dotenv import load_dotenv
import os

# Change "keys.env" to ".env" if that's your filename
load_dotenv("keys.env")

def masked(v): 
    return v[:4] + "..." + v[-4:] if v and len(v) >= 8 else str(bool(v))

print("GEMINI_API_KEY:", masked(os.getenv("GEMINI_API_KEY")))
assert os.getenv("GEMINI_API_KEY"), "Missing GEMINI_API_KEY in keys.env/.env"

GEMINI_API_KEY: AIza...2-E8


In [11]:
from google import genai
from google.genai import types
import sys

# --- Setup ---
client = genai.Client()
MODEL = "gemini-2.5-flash"

# Create a chat session (keeps history across turns)
chat = client.chats.create(
    model=MODEL,
    config=types.GenerateContentConfig(
        temperature=0.0,
        max_output_tokens=128,
        candidate_count=1,
    )
)

# --- Helpers ---
def _extract_text_from_response(resp) -> str:
    """Pull full text from a non-streaming response."""
    if not getattr(resp, "candidates", None):
        return ""
    out = []
    for cand in resp.candidates or []:
        content = getattr(cand, "content", None)
        parts = getattr(content, "parts", []) if content else []
        for p in parts:
            t = getattr(p, "text", None)
            if t:
                out.append(t)
    return "".join(out)

def _extract_text_from_event(event) -> str:
    """Pull any text from a streaming event."""
    out = []
    for cand in getattr(event, "candidates", []) or []:
        content = getattr(cand, "content", None)
        parts = getattr(content, "parts", []) if content else []
        for p in parts:
            t = getattr(p, "text", None)
            if t:
                out.append(t)
    return "".join(out)

def _maybe_print_safety(event):
    """Show safety blocks if present."""
    fb = getattr(event, "prompt_feedback", None)
    if not fb:
        return
    block = getattr(fb, "block_reason", None) or getattr(fb, "block_reason_message", None)
    if block:
        print(f"\n[blocked: {block}]", flush=True)

# --- Main ask function ---
def ask(user_text: str):
    """Stream reply token by token; fallback to non-streaming if needed."""
    print(f"\nYou: {user_text}")
    print("Model:", end=" ", flush=True)

    got_text = False
    try:
        for event in chat.send_message_stream(user_text):
            chunk = _extract_text_from_event(event)
            if chunk:
                print(chunk, end="", flush=True)
                got_text = True
            else:
                _maybe_print_safety(event)
    except Exception as e:
        print(f"\n[stream error] {e}", file=sys.stderr)

    print()  # newline after stream

    # fallback if nothing visible came out
    if not got_text:
        try:
            resp = chat.send_message(user_text)
            text = _extract_text_from_response(resp)
            if text:
                print(text)
            else:
                print("[no text returned; possibly safety-filtered or empty]")
        except Exception as e:
            print(f"[fallback error] {e}")

# --- Example REPL loop ---
if __name__ == "__main__":
    while True:
        q = input("\n> ")
        if q.strip().lower() in {"exit", "quit"}:
            break
        ask(q)


KeyboardInterrupt: Interrupted by user

In [12]:
from google import genai
from google.genai import types
import sys

# ---------- Setup ----------
client = genai.Client()
MODEL = "gemini-2.5-flash"

chat = client.chats.create(
    model=MODEL,
    config=types.GenerateContentConfig(
        temperature=0.0,
        max_output_tokens=128,
        candidate_count=1,
    )
)

# ---------- Robust extractors ----------
def _parts_text(parts):
    """Join any available .text in a parts list; tolerate missing fields."""
    out = []
    if not parts:
        return ""
    for p in parts:
        t = getattr(p, "text", None)
        if t:
            out.append(t)
    return "".join(out)

def _extract_text_from_response(resp) -> str:
    """Non-streaming: safely extract all text from resp."""
    if resp is None:
        return ""
    cands = getattr(resp, "candidates", None) or []
    out = []
    for cand in cands:
        content = getattr(cand, "content", None)
        parts = getattr(content, "parts", None)
        out.append(_parts_text(parts))
    return "".join(out)

def _extract_text_from_event(event) -> str:
    """Streaming: safely extract any text from a stream event."""
    if event is None:
        return ""
    cands = getattr(event, "candidates", None) or []
    out = []
    for cand in cands:
        content = getattr(cand, "content", None)
        parts = getattr(content, "parts", None)
        out.append(_parts_text(parts))
    return "".join(out)

def _maybe_print_safety(event):
    fb = getattr(event, "prompt_feedback", None)
    if not fb:
        return
    # try a couple of common attribute names
    reason = getattr(fb, "block_reason", None) or getattr(fb, "block_reason_message", None)
    if reason:
        print(f"\n[blocked: {reason}]", flush=True)

# ---------- Main: streaming with fallback ----------
def ask(user_text: str):
    """Stream the reply; if stream yields no visible text, fallback to non-streaming."""
    print(f"\nYou: {user_text}")
    print("Model:", end=" ", flush=True)

    got_text = False
    try:
        for event in chat.send_message_stream(user_text):
            chunk = _extract_text_from_event(event)
            if chunk:
                print(chunk, end="", flush=True)
                got_text = True
            else:
                _maybe_print_safety(event)
    except Exception as e:
        print(f"\n[stream error] {e}", file=sys.stderr)

    print()  # newline after stream

    # Fallback: call non-streaming once if nothing was printed
    if not got_text:
        try:
            resp = chat.send_message(user_text)
            text = _extract_text_from_response(resp)
            if text:
                print(text)
            else:
                print("[no text returned; possibly safety-filtered or empty]")
        except Exception as e:
            print(f"[fallback error] {e}")

# ---------- Optional: notebook-friendly helper ----------
def ask_once(user_text: str) -> str:
    """Return the full text in one go (no streaming) — handy in notebooks."""
    try:
        resp = chat.send_message(user_text)
        return _extract_text_from_response(resp)
    except Exception as e:
        return f"[error] {e}"

# ---------- REPL (comment out when using in notebooks) ----------
if __name__ == "__main__":
    while True:
        q = input("\n> ")
        if q.strip().lower() in {"exit", "quit"}:
            break
        ask(q)



You: hi
Model: Hi there! How can I help you today?

You: iam doing real time convestation with ai project can you help me
Model: 
[no text returned; possibly safety-filtered or empty]

You: ...............................
Model: 
[no text returned; possibly safety-filtered or empty]

You: Hi there! How can I help you today?
Model: 
[no text returned; possibly safety-filtered or empty]

You: 
Model: 
It looks like you might have sent an empty message.

Is there something you'd like to talk about or ask? I'm here to help!


: 